In [1]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns

from amftrack.util.dbx import upload_folders, load_dbx, download
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import (
    read_saved_dropbox_state,
    get_dropbox_folders_prince,
    get_dropbox_folders_general,
    save_dropbox_state,
    get_dropbox_folders_general_recursive,
)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
all_folders_drop = get_dropbox_folders_prince("/DATA/MYRISTATE/PRINCE")
# all_folders_drop = get_dropbox_folders_prince("/DATA/PRINCE")

2024-04-25 09:42:45,992-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:46,237-[INFO]- dropbox:474 -> Request to files/list_folder


sizes:   0%|          | 0/118 [00:00<?, ?it/s]

2024-04-25 09:42:46,752-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:46,970-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:47,291-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:47,510-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:47,810-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:48,011-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:48,308-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:48,510-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:49,009-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:49,259-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:49,681-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:49,950-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-25 09:42:50,407-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-25 09:42:50,694-[INFO]- dropbox:

In [13]:
plates = [
    "63_20230510",
    "34_20230221",
    "61_20230510",
    "64_20230508",
    "33_20230221",
    "35_20230227",
]
plates = [
    # "62_20230511",
    # "49_20230510",
    # "60_20230508",
    "68_20230501",
    # "14_20230301",
    # "58_20230511",
    # "1_20230221",
    # "51_20230511",
    # "48_20230301",
    # "33_20230221",
    # "46_20230221",
]
folders_drop = all_folders_drop[all_folders_drop["unique_id"].isin(plates)]
folders_drop = folders_drop[folders_drop['folder']=="20230523_2252_Plate13"]

In [14]:
folders_drop

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
6201,20230523_2252_Plate13,NaN,0,data/myristate/prince/68_20230501/20230523_225...,68,20230501,68_20230501


In [ ]:
all_folders_drop[all_folders_drop["unique_id"] == "61_20230510"]

In [ ]:
data = all_folders_drop.groupby("unique_id")["folder"].count()

In [ ]:
plates = [
    1,
    49,
    51,
    14,
    58,
    59,
    60,
    31,
    33,
    34,
    62,
    63,
    64,
    67,
    68,
    69,
    72,
    37,
    46,
    48,
]

In [ ]:
plates = [str(plate) for plate in plates]

In [ ]:
folders = all_folders_drop[all_folders_drop["Plate"].isin(plates)]
folders_drop = folders[folders["CrossDate"] > "20230101"]

In [ ]:
folders_drop.groupby("unique_id")["folder"].count()

In [ ]:
plates = [
    "58_20230511",
    "62_20230511",
    "1_20230221",
    "63_20230510",
    "51_20230510",
    "49_20230510",
    "64_20230508",
    "60_20230508",
    "59_20230508",
    "72_20230501",
    "69_20230501",
    "67_20230501",
    "68_20230501",
    "37_20230221",
    "14_20230301",
    "34_20230227",
    "31_20230227",
    "48_20230227",
    "33_20230221",
    "46_20230221",
]
plates = [
    "1_20230221",
    "51_20230510",
    "49_20230510",
    "64_20230508",
    "72_20230501",
    "69_20230501",
    "67_20230501",
    "34_20230227",
    "31_20230227",
    "33_20230221",
]

In [ ]:
folders_drop = all_folders_drop[all_folders_drop["unique_id"].isin(plates)]

In [ ]:
folders_drop.groupby("unique_id")["folder"].count()

In [ ]:
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(all_folders["folder"])]
folders_drop2

In [16]:
# directory_targ = directory_project + "myr/"
directory_targ = os.path.join(directory_scratch, "myr") + "/"
# directory_targ = directory_targ+'myr/'
directory_targ

'/scratch-shared/amftrack/myr/'

In [17]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    40,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="fig1",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 6041787


In [3]:
update_plate_info(directory_targ, local=True, strong_constraint=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/1013 [00:00<?, ?it/s]

In [ ]:
plates = all_folders["unique_id"].unique()
plates = plates[:5]

In [ ]:
folders = all_folders[all_folders["unique_id"].isin(plates)]

In [4]:
folders = all_folders

In [ ]:
plates = [
    "63_20230510",
    "34_20230221",
    "61_20230510",
    "64_20230508",
    "33_20230221",
    "35_20230227",
]
plates += [
    "62_20230511",
    "49_20230510",
    "60_20230508",
    "68_20230501",
    "14_20230301",
    "58_20230511",
    "1_20230221",
    "51_20230511",
    "48_20230301",
    "33_20230221",
    "46_20230221",
]
folders = all_folders[all_folders["unique_id"].isin(plates)]

In [4]:
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    all_folders,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="correct_myr_files",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 6037534


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 6037535


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


...

Submitted batch job 6037552


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 6037553


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 6037554


To correct maforme Img paths

In [ ]:
update_plate_info(directory_targ, local=True, strong_constraint=False)
wrong_folder = get_current_folders(directory_targ, local=True)

In [ ]:
wrong_folder

In [ ]:
import shutil

for folder_path in wrong_folder["total_path"]:

    # Check if it's a directory and contains an 'Img' subfolder
    imgimg_subfolder_path = os.path.join(folder_path, "Img", "Img")
    if os.path.isdir(folder_path) and os.path.isdir(imgimg_subfolder_path):

        # Move each file from the 'Img' subfolder to its parent folder
        for file in os.listdir(imgimg_subfolder_path):
            final_file_path = os.path.join(folder_path, "Img", file)
            init_file_path = os.path.join(imgimg_subfolder_path, file)
            print(init_file_path, final_file_path)
            shutil.move(init_file_path, final_file_path)

        # Remove the now empty 'Img' subfolder
        os.rmdir(imgimg_subfolder_path)